# Initialization
## Start by installing requried packages

In [ ]:
!pip3 install openai --quiet

## Previewing Data

Data should be loaded into a table called `embeddings`. Let's preview that data to see what we're working with.

In [ ]:
%%sql
select * from embeddings limit 2;

In [13]:
%%sql
select count(*) from embeddings;

count(*)
4403


## Building the vector indexes

Let's build our vector index. SingleStore gives us many options for our index with many tunable parameters.

![annvknn](https://github.com/bscolinos/vector_workshop/raw/main/screenshots/knn_v_ann.png)

In [ ]:
%%sql
alter table embeddings add vector index auto (v) INDEX_OPTIONS '{"index_type":"AUTO"}';

### IVF_PQ

Starting with an IVF_PQ algorithm. Pros: fast build time, low memory usage. Cons: Lower recall than HNSW, slower than HNSW in production

In [ ]:
%%sql
alter table embeddings add vector index ivf_pq (v) INDEX_OPTIONS '{"index_type":"IVF_PQ"}';

### HNSW_FLAT

Pros: Great recall (similar performance to kNN), great runtime on queries. Cons: Memory intensive. Potentially slow build time

In [ ]:
%%sql
alter table embeddings add vector index hnsw_flat (v) INDEX_OPTIONS '{"index_type":"HNSW_FLAT"}';

### Syntax to drop indexes if needed

In [ ]:
%%sql
drop index auto on embeddings;

## Testing our vector indexes

Compare the performance of an exact K-nearest neighbor search to the searches with our ANN indexes.

Due to the relatively small size of the dataset, the search times aren't drastically different, but the increase in speed becomes exponential as the size of the dataset grows.

In [155]:
%%sql
set @qv = (select v from embeddings where id = 2251799813685522);

-- NO INDEX: exact kNN search
select content, v <*> @qv as sim
from embeddings
order by sim use index () desc
limit 3;

content,sim
"ing model predictions that maximized the true positive rate while minimizingthe false positive rate across the test set. Each GradCAM heatmap was firstconverted to a binary mask by thresholding at the lowest non-zero value of theGrad-CAM heatmap. 5 features were used to generate each LIME mask. For ev-ery shape in the image, we calculated the intersection over union (IOU) betweenthe shape and the explanation. Finally, we ranked shapes by their mean IOUacross the entire test set.DEPICT: Diffusion-Enabled Permutation Importance 19""green circle 7(110,16) blue circle23 (44,59) greenrectangle ((87, 70),(175, 210)) bluerectangle ((38,104), (55, 183))""Caption Real Generated""blue circle 27(227,77) redrectangle ((9, 58),(112, 167)) greenrectangle ((126,106), (178, 237))""""red rectangle ((123,22), (248, 87))""Fig.10: Comparison between real and generated images for syntheticdataset. We compare real and generated images from the diffusion model conditioned",1.0
"the false positive rate of the validation set. Each GradCAM heatmap was firstconverted to a binary mask by thresholding at the lowest non-zero value of theGrad-CAM heatmap. 5 features were used to generate each LIME mask. For ev-ery object in the image, we calculated the intersection over union (IOU) betweenthe object mask and the explanation. Finally, we ranked objects by their meanIOU across the entire test set.Unconstrained primary feature models. In reality, we might want to ex-plain a model that is not a concept bottleneck. Thus, we also trained primaryfeature models end-to-end. When the model is not constrained to a specific setof concepts, we want to observe that DEPICT still detects the primary featureas the most important concept in a classifier’s decisions.9.2 ResultsUnconstrained primary feature models. We compare three randomly se-lected unconstrained (trained end-to-end) primary feature model rankings gen-",0.9171483516693115
"featuremodels that rely on either “person” or “tv” when predicting home or hotel .When permuting the most important concept, model performance is low, whereas whenpermuting concept that the model does not rely on, model performance does not drop.based explanations, we extend these approaches to generate a ranking by relyingon concept annotations and their corresponding mask. Because we have access tothe image generation process of the synthetic dataset, we generate an concept-level mask for all concepts in each image. Then, for each image, we calculate theintersection-over-union (IOU) between each concept-level mask and the Grad-CAM or LIME mask generated by the classifier (full details are in supplementary8). Then, we rank concepts by their mean IOU across the entire test set. We notethat computing this ranking for GradCAM and LIME requires access to image-level masks as well as the model parameters, while DEPICT does not. Because",0.8815749287605286


In [ ]:
%%sql
set @qv = (select v from embeddings where id = 2251799813685522);


select content, v <*> @qv as sim
from embeddings
order by sim use index (auto) desc
limit 3;

In [ ]:
%%sql
set @qv = (select v from embeddings where id = 2251799813685522);

-- IVF_PQ
select content, v <*> @qv as sim
from embeddings
order by sim use index (ivf_pq) desc
limit 3;

In [ ]:
%%sql
set @qv = (select v from embeddings where id = 2251799813685522);

-- HNSW_FLAT
select content, v <*> @qv as sim
from embeddings
order by sim use index (hnsw_flat) desc
limit 3;

## Building the fulltext indexes

Now we'll build our fulltext index. As of version 8.7, SingleStore supports Exact phrase search, Fuzzy search, BM25 scoring, Proximity search, Boosting, Wildcard searches, Boolean operators, JSON search via the JLucene library

Additional information on these can be found here: https://docs.singlestore.com/cloud/developer-resources/functional-extensions/working-with-full-text-search/

In [ ]:
%%sql
alter table embeddings add fulltext using VERSION 2 fts (content);

## Using the fulltext indexes

Once the fulltext index is built, we can test the various search options. In this example, we'll demonstrate fuzzy matching and a BM25 search

In [3]:
%%sql
SELECT id, content, 
       MATCH(TABLE embeddings) AGAINST('content:singlestor~ content:datbase~') AS score
FROM embeddings
WHERE MATCH(TABLE embeddings) AGAINST('content:singlestor~ content:datbase~')
ORDER BY score DESC
LIMIT 3;

id,content,score
2251799813688675,"propriate content (e.g., regex for personal identifiers). Fur-thermore, we fine-tune the ELECTRA model (Clark et al.,3Open Artificial Knowledge Dataset2020) on publicly available toxicity datasets (Sorensen et al.,2017) to provide a filtering score.For the Evaluation Metrics (C8) challenge, we will engagethe community to fine-tune an LLM on the OAK dataset andevaluate it using common benchmarks such as WinoGrande(Sakaguchi et al., 2021), ARC Easy (Clark et al., 2018), andso forth, the list of the benchmarks will be extended.We plan regular updates of the OAK dataset to reflect newtrends and information, addressing the Maintenance andUpdate of Synthetic Data (C10). This step ensures thedataset remains relevant and effective for training purposes.The data generation pipeline is illustrated in Fig. 1, startingwith the querying of knowledge databases to gather topics,which are then expanded using LLMs. These topics are",2.03981351852417
2251799813687603,"about the extracted code was included, encompassing PL specifications, code complex-ity metrics, and more [9]. For this work, version 1.0.7 of the CVEFixes dataset2 was used , which is the most recent version available at the time of writing. Notably, this version incorporates an increased amount of data associated with C/C++ code compared to its predecessors. To use the code within this dataset, it was necessary to extract it from the database. This involved first creating indexes on columns containing identifiers to reduce processing time, as without this optimization, query execution times were too long. For our analy-sis, the code at function level was used along with their respective labels, so o nce the indexes were created, the query presented in Fig. 1 was executed to extract the relevant data from the dataset . Fig. 1. SQL query used to extract data from the dataset",2.0000267028808594
2251799813688698,"high-quality synthetic dataset, forming the foundation for the subsequent steps in the OAK dataset generation pipeline.The methodology for Subtopic Expansion involves several key steps: Initially, high-level topics are extracted from extensivehuman knowledge databases such as Wikipedia. This ensures a broad and diverse range of starting points for subtopic8Open Artificial Knowledge Dataset# List of interest areasinterest_areas = [""technology"", ""history"", ""art"", ""science"", ...]def generate_prompt(categories, page_counts):interest = choose_random(categories, page_counts)ifrandom_chance(0.09):random_title = fetch_random_wikipedia_title()else :page_titles = search_wikipedia(interest, 5)random_title = choose_random(page_titles)summary = fetch_wikipedia_summary(random_title, 2)analysis_type = choose_random([""summarize"", ""provide an in-depth analysis of"", ""contrast with another topic inthe same field"",",1.9422978162765503


In [4]:
%%sql
SELECT id, content, ROUND(0.8*BM25(embeddings,'content:markov') + 0.4*BM25(embeddings,'content:intelligence'),3) AS score_boosted
FROM embeddings
ORDER BY score_boosted DESC
LIMIT 3;

id,content,score_boosted
2251799813686900,"whether we should include that in the generalization bound, since Ris indeed generated based on Z,corresponding to the true Markov chain: S→Z→R.To clarify, the Markov chain for the training without rationale is: S→Z→R, with additionalarrows Z→θunandS→θun, but no arrow from Rtoθun.Intuitively, we should account for this difference by arguing that, conditioned on the preference Z,the learned model θunis conditionally independent of R. However, due to this difference, it seemsprudent to reason from first principles.Let’s start by choosing the distributions PandQfor the Donsker-Varadhan variational representationof the KL divergence. We set P=PS,R,Z,θ unandQ=µn⊗Pθun, where µis the distribution for17(S, R, Z ). Then, for any measurable function f, we have:D(P∥Q)≥EP[f(S, R, Z, θ )]−logE(¯D,¯R,¯Z,¯θ)∼Q[ef(¯S,¯R,¯Z,¯θ)]. (7)Now, choose f(S, R, Z, θ ) =λ(ℓD(θ)−ℓµ(θ))for some λ∈R, where ℓD(θ)is the empirical loss",2.332
2251799813686071,"of applying LLMs to controllable and hybrid planning, optimizing for both accuracy and efficiency.This, in turn, allows System- 1.xto outperform both System- 1and System- 2at a fixed budget. Byintelligently allocating more resources to harder sub-goals, System- 1.xis able to save its System- 2budget for cases where it is necessary.2 System- 1.x: Controllable and Hybrid Planning with LLMsWe start with the requisite background on planning problems and our setup (§2.1), followed by anoverview and a detailed description of our method System- 1.xPlanner in §2.2 and §2.3, respectively.2.1 Background and Problem SetupA planning problem can be modeled as a Markov Decision Process M= (S,A,T,R)defined witha set of states S, a set of actions Aon any given state, a transition function T:S ×A → S definingtransitions between pairs of states based on an action, and a reward function R:S →Rassigning",2.132
2251799813689499,"DKL(Pt∥Q)will shrink to zero.3.2. RLHF with KL penalty under heavy-tailed returndistributionWe now adapt our result to the case where the policy is alanguage model and we are training it using RLHF. We arenow applying KL divergence over the policies rather thanthe return distributions. We first formally define the prop-erties of RLHF on language models that cause the result tohold: namely, when when considered as a Markov decisionprocess (MDP), environmental transitions are deterministicand return depends only on the final state reached.Definition: A deterministic-transition MDP with Markovianreturns (DMRMDP) is an MDP (S,A, P, R )such that:•The transition function P:S×A → S is deterministic,i.e., for each state s∈ S and action a∈ A, there existsa unique state s′∈ Ssuch that P(s′|s, a) = 1 .In RLHF: the transition is appending the generatedtoken ato the context s.•There is a set of sink states E⊆ S that terminate a",2.092


## Hybrid Search in SingleStore

Implementing a "hybrid search" in SingleStore. This is going to be a query that combines two powerful tools: a fulltext search and a semantic search

In [163]:
%%sql
SET @qv = (SELECT v FROM embeddings WHERE id = 2251799813685522);

WITH max_bm25 AS (
    SELECT MAX(0.8 * BM25(embeddings, 'content:markov') + 0.4 * BM25(embeddings, 'content:intelligence')) AS max_score
    FROM embeddings
)
SELECT 
    e.id,
    e.content,
    ROUND(
        (0.7 * (e.v <*> @qv)) + 
        (0.3 * (0.8 * BM25(e, 'content:markov') + 0.4 * BM25(e, 'content:intelligence')) / max_bm25.max_score),
    3) AS hybrid_score
FROM 
    embeddings e
CROSS JOIN max_bm25
ORDER BY 
    hybrid_score DESC
LIMIT 3;

id,content,hybrid_score
2251799813686900,"whether we should include that in the generalization bound, since Ris indeed generated based on Z,corresponding to the true Markov chain: S→Z→R.To clarify, the Markov chain for the training without rationale is: S→Z→R, with additionalarrows Z→θunandS→θun, but no arrow from Rtoθun.Intuitively, we should account for this difference by arguing that, conditioned on the preference Z,the learned model θunis conditionally independent of R. However, due to this difference, it seemsprudent to reason from first principles.Let’s start by choosing the distributions PandQfor the Donsker-Varadhan variational representationof the KL divergence. We set P=PS,R,Z,θ unandQ=µn⊗Pθun, where µis the distribution for17(S, R, Z ). Then, for any measurable function f, we have:D(P∥Q)≥EP[f(S, R, Z, θ )]−logE(¯D,¯R,¯Z,¯θ)∼Q[ef(¯S,¯R,¯Z,¯θ)]. (7)Now, choose f(S, R, Z, θ ) =λ(ℓD(θ)−ℓµ(θ))for some λ∈R, where ℓD(θ)is the empirical loss",0.819
2251799813686071,"of applying LLMs to controllable and hybrid planning, optimizing for both accuracy and efficiency.This, in turn, allows System- 1.xto outperform both System- 1and System- 2at a fixed budget. Byintelligently allocating more resources to harder sub-goals, System- 1.xis able to save its System- 2budget for cases where it is necessary.2 System- 1.x: Controllable and Hybrid Planning with LLMsWe start with the requisite background on planning problems and our setup (§2.1), followed by anoverview and a detailed description of our method System- 1.xPlanner in §2.2 and §2.3, respectively.2.1 Background and Problem SetupA planning problem can be modeled as a Markov Decision Process M= (S,A,T,R)defined witha set of states S, a set of actions Aon any given state, a transition function T:S ×A → S definingtransitions between pairs of states based on an action, and a reward function R:S →Rassigning",0.787
2251799813689330,"probability of transitioning between the different tokens, γtdenotes the prob-ability of transitioning to a token, and αt= 1−Kβt−γt. Due to the6 S. Chi et al.Markov property, the probabilities of ztat arbitrary diffusion time step can bederived q(zt|z0) =v⊤(zt)Qtv(z0), where Qt=QtQt−1···Q1. The matrix isconstructed such that the token always maintains its original state sothatztconverges to token with sufficiently large t.Conditional Denoising Process. The conditional denoising process througha neural network pθ. This network predicts the noiseless token z0when providedwith a corrupted token and its corresponding condition, such as a languagetoken. For training the network pθ, beyond the denoising objective, the trainingincorporates the standard variational lower bound objective [50], denoted asLvlb. The training objective with a coefficient for the denoising loss λis:L=Lvlb+λEzt∼q(zt|z0)[−logpθ(z0|zt, y)], (3)",0.785


## RAG in 3 functions

We'll test both open and closed source models to compare responses / usability

![rag](https://github.com/bscolinos/vector_workshop/raw/main/screenshots/rag_architecture.png)

In [ ]:
import sqlalchemy as sa
from openai import OpenAI
import os
import json

In [ ]:
from singlestoredb.management import get_secret

OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] = get_secret('OPENAI_API_KEY')
client = OpenAI()
EMBEDDING_MODEL = 'text-embedding-ada-002'
GPT_MODEL = 'gpt-3.5-turbo'

# SingleStore connection
engine = sa.create_engine(connection_url)
connection = engine.connect()

In [ ]:
def get_embedding(text, model=EMBEDDING_MODEL):
    '''Generates the OpenAI embedding from an input `text`.'''
    if isinstance(text, str):
        response = client.embeddings.create(input=[text], model=model)
        return json.dumps(response.data[0].embedding)

In [ ]:
def vector_search(query, limit=15):
    '''Returns a df of the top k matches to the query ordered by similarity.'''
    query_embedding_vec = get_embedding(query)
    statement = sa.text(
        f'''select content, v <*> :query_embedding :> vector(1536) AS similarity
        from embeddings
        order by similarity use index () desc
        limit :limit;'''
    )
    results = connection.execute(statement, {"query_embedding": query_embedding_vec, "limit": limit})
    results_as_dict = results.fetchall()
    return results_as_dict

In [ ]:
def rag(query, limit=5, temp=0.1):
    '''Uses RAG to answer a question from the wiki page'''
    results = vector_search(query, limit)
    print("Asking Chatbot...")
    prompt = f'''Excerpt from the conversation history:
        {results}
        Question: {query}

        You are a research assistant. Based on the conversation history, try to provide the most accurate answer to the question.
        Consider the details mentioned in the conversation history to formulate a response that is as
        helpful and precise as possible. 

        '''
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "You are a research assistant who is answering questions about an article."},
            {"role": "user", "content": prompt}
        ],
        temperature=temp
    )
    response_message = response.choices[0].message.content
    
    return response_message

In [ ]:
query = "What is a markov chain?"
rag(query)